LangChain Implementation of Chain of Verification (CoVe) to reduce hallucination in LLM

In [ ]:
!pip install langchain openai langchain_community langchain-openai

In [ ]:
import os
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain, LLMChain
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [ ]:
from google.colab import userdata

In [ ]:
userdata.get('OPENAI_API_KEY')

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(model_name="gpt-5-nano")

query = "List 5 UK politicians born in London"

Generate baseline response

In [ ]:
input_variables = ["query"]
base_response_output_key = "base_response"
base_response_template = """Question: {query} Answer:"""
base_response_prompt_template = PromptTemplate(
    input_variables=input_variables, template=base_response_template
)
base_reponse_chain = LLMChain(
    llm=llm, prompt=base_response_prompt_template,
    output_key=base_response_output_key
)

Plan verification

In [ ]:
plan_verifications_template = """
Given the below Question and answer, generate a series of verification questions that test the factual claims in the original baseline response.
For example if part of a longform model response contains the statement “The Mexican–American War
was an armed conflict between the United States and Mexico from 1846 to 1848”, then one possible
verification question to check those dates could be “When did the Mexican American war start and
end?”

Question: {query}
Answer: {base_response}

<fact in passage>, <verification question, generated by combining the query and the fact>

{format_instructions}
"""

class PlanVerificationsOutput(BaseModel):
    query: str = Field(description="The user's query")
    base_response: str = Field(description="The response to the user's query")
    facts_and_verification_questions: dict[str, str] = Field(
        description="Facts (as the dictionary keys) extracted from the response and verification questions related to the query (as the dictionary values)"
    )

plan_verifications_output_parser = PydanticOutputParser(
    pydantic_object=PlanVerificationsOutput
)

plan_verifications_prompt_template = PromptTemplate(
    input_variables=input_variables + [base_response_output_key],
    template=plan_verifications_template,
    partial_variables={
        "format_instructions": plan_verifications_output_parser.get_format_instructions()
    },
)

plan_verifications_chain = LLMChain(
    llm=llm,
    prompt=plan_verifications_prompt_template,
    output_key="output",
    output_parser=plan_verifications_output_parser,
)

The 2 chains (base_response_chain and plan_verifications_chain can then be run sequentially

```
# This is formatted as code
```



In [ ]:
answer_and_plan_verification = SequentialChain(
    chains=[base_reponse_chain, plan_verifications_chain],
    input_variables=["query"],
    output_variables=["output"],
    verbose=True
)
intermediate_result = answer_and_plan_verification.invoke(query)

Generate verification chain

In [ ]:
# Define Pydantic model for verification output
class VerificationOutput(BaseModel):
    fact: str = Field(description="The fact being verified")
    verification_question: str = Field(description="The question used to verify the fact")
    answer: str = Field(description="The answer to the verification question from the LLM")
    is_consistent: bool = Field(description="Whether the answer is consistent with the original fact (True/False)")
    inconsistency_reason: str | None = Field(
        description="Reason for inconsistency, if applicable"
    )

# Create PydanticOutputParser for verification output
verification_output_parser = PydanticOutputParser(pydantic_object=VerificationOutput)

# Define verification template
verification_template = """
Given the context and the fact, answer the verification question and indicate if the answer is consistent with the original fact.

Context: {base_response}
Fact: {fact}
Verification Question: {verification_question}

Answer: {answer}
Is Consistent: {is_consistent}
Reason for inconsistency: {inconsistency_reason}

{format_instructions}
"""

# Create PromptTemplate for verification
verification_prompt_template = PromptTemplate(
    input_variables=["base_response", "fact", "verification_question"],
    template=verification_template,
    partial_variables={
        "format_instructions": verification_output_parser.get_format_instructions()
    },
)

# Create LLMChain for verification
verification_chain = LLMChain(
    llm=llm,
    prompt=verification_prompt_template,
    output_key="verification_output",
    output_parser=verification_output_parser,
)